# 01 - Interview Question Generation
`Author: Abdlazeez Jimoh`

In [3]:
from openai import OpenAI
import json
from pprint import pprint
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [4]:
client = OpenAI()

In [5]:
class BaseAgent:
    def __init__(self):
        self.client = OpenAI()
        self.system_prompt = None
        self.user_prompt = None

    def __call__(self, *args, **kwargs):
        raise NotImplementedError


class ResponseEvaluationAgentV1(BaseAgent):
    def __init__(self):
        super().__init__()
        self.system_prompt = """You are an interviewer evaluating a candidate's response to an interview question. Your task is to:
- Evaluate the candidate's response on the scale of "good", "average", and "bad".
- Provide a reason for why it's categorized as good, average, or bad.
- Offer constructive feedback or suggestions for improvement.
- Provide two samples of good responses.

You will be provided with an interview question and a candidate response.

Evaluate and provide output in the following JSON format:
{
    "evaluation": "good, average, or bad",
    "reason": "Reason why it's good, average, or bad",
    "feedback": "Feedback or suggestions for improvement",
    "sample": [
        "Good sample response 1",
        "Good sample response 2"
    ]
}"""
        self.user_prompt = """QUESTION:
{question} 
RESPONSE: {response}"""

    def __call__(self, question: str, response: str) -> dict[str, str]:
        """
        Evaluate a candidate's response to an interview question.

        Args:
            question (str): The interview question.
            response (str): The candidate's response.

        Returns:
            dict: The evaluation of the candidate's response.
        """

        # Generate questions
        evaluation = self._generate(question, response)

        return evaluation

    def run(self, question: str, response: str) -> dict[str, str]:
        """
        Evaluate a candidate's response to an interview question.

        Args:
            question (str): The interview question.
            response (str): The candidate's response.

        Returns:
            dict: The evaluation of the candidate's response.
        """

        # Generate questions
        evaluation = self._generate(question, response)

        return evaluation

    def _generate(self, question: str, response: str) -> dict[str, str]:
        """
        Evaluate a candidate's response to an interview question.

        Args:
            question (str): The interview question.
            response (str): The candidate's response.

        Returns:
            dict: The evaluation of the candidate's response.
        """

        try:
            output = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {
                        "role": "system",
                        "content": self.system_prompt,
                    },
                    {
                        "role": "user",
                        "content": self.user_prompt.format(question=question, response=response),
                    },
                ],
                temperature=0.5,
                max_tokens=1024,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
            )

            questions = json.loads(output.choices[0].message.content or "{}")

            return questions
        except Exception as e:
            print(e)
            return {}



In [6]:
response_evaluator = ResponseEvaluationAgentV1()
evaluation = response_evaluator.run(
    "What motivated you to pursue a career in software engineering at a startup in San Francisco?",
    "I've always been interested in technology and startups. I've been working in the tech industry for the past 5 years and I've always wanted to work at a startup. I'm really excited about the opportunity to work at a startup in San Francisco."
)

pprint(evaluation)

{'evaluation': 'average',
 'feedback': 'The candidate should try to add more depth to their response by '
             'sharing specific experiences or projects that fueled their '
             'interest in technology and startups. It would also be beneficial '
             "to mention how the candidate's skills and goals align with the "
             'startup environment.',
 'reason': "The response provides a basic explanation of the candidate's "
           'interest in technology and startups, but it lacks specific details '
           'or personal anecdotes to make it stand out.',
 'sample': ['One of the pivotal moments that inspired me to pursue a career in '
            'software engineering was when I developed a mobile app for a '
            'small business during my college years. Witnessing the impact of '
            'technology on a growing venture fueled my passion for creating '
            'innovative solutions and collaborating with like-minded '
            'individua